In [45]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ubuntu/varios/skforecast'

In [46]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn


import numpy as np
import pandas as pd
import time
import warnings
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
%load_ext pyinstrument 

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument


## Information system and libraries

In [47]:
# Versions
# ==============================================================================
print(f"Python version: {platform.python_version()}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"skforecast version: {skforecast.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scipy version: {scipy.__version__}")
print(f"psutil version: {psutil.__version__}")
print("")

# Computer information
# ==============================================================================
#Computer network name
print(f"Computer network name: {platform.node()}")
#Machine type
print(f"Machine type: {platform.machine()}")
#Processor type
print(f"Processor type: {platform.processor()}")
#Platform type
print(f"Platform type: {platform.platform()}")
#Operating system
print(f"Operating system: {platform.system()}")
#Operating system release
print(f"Operating system release: {platform.release()}")
#Operating system version
print(f"Operating system version: {platform.version()}")
#Physical cores
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
#Logical cores
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.10.9
scikit-learn version: 1.2.1
skforecast version: 0.9.0
pandas version: 2.0.2
numpy version: 1.23.5
scipy version: 1.10.1
psutil version: 5.9.0

Computer network name: ip-10-2-1-218
Machine type: x86_64
Processor type: x86_64
Platform type: Linux-5.15.0-1037-aws-x86_64-with-glibc2.31
Operating system: Linux
Operating system release: 5.15.0-1037-aws
Operating system version: #41~20.04.1-Ubuntu SMP Mon May 22 18:18:00 UTC 2023
Number of physical cores: 4
Number of logical cores: 8


## Data

In [48]:
# Data
# ==============================================================================
n = 10_000
rgn = np.random.default_rng(seed=123)
y = pd.Series(rgn.random(size=(n)), name="y")
exog = pd.DataFrame(rgn.random(size=(n, 10)))
exog.columns = [f"col_{i}" for i in range(exog.shape[1])]
y_train = y[:-int(n/2)]
display(y.head())
display(exog.head())   

0    0.682352
1    0.053821
2    0.220360
3    0.184372
4    0.175906
Name: y, dtype: float64

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
0,0.855557,0.528149,0.611974,0.809502,0.999681,0.261925,0.480293,0.361009,0.843979,0.596611
1,0.002650,0.705236,0.547129,0.819472,0.705629,0.191310,0.617129,0.441121,0.914477,0.438860
2,0.392213,0.349656,0.258877,0.920072,0.427726,0.032372,0.500220,0.929874,0.277166,0.961383
3,0.289545,0.150200,0.214419,0.014987,0.574105,0.562479,0.631097,0.765738,0.863536,0.562381
4,0.999299,0.077688,0.868944,0.913900,0.158110,0.145379,0.334003,0.429158,0.236270,0.326537


## Profiling ForecasterAutoreg

In [49]:
warnings.filterwarnings("ignore")

In [50]:
# Forecaster
# ==============================================================================
print("-----------------")
print("ForecasterAutoreg")
print("-----------------")


regressors = [
        Ridge(random_state=36987),
        RandomForestRegressor(random_state=36987),
        LGBMRegressor(random_state=36987),
        HistGradientBoostingRegressor(random_state=36987)
]
param_grids = [
        {'alpha': [0.1, 1, 10]},
        {'n_estimators': [5], 'max_depth': [5, 10, 15]},
        {'max_iter': [50, 200], 'max_depth': [5, 10, 15]},
        {'max_iter': [50, 200], 'max_depth': [5, 10, 15]}
]
lags_grid = [50, 50, 50]

elapsed_times = []
methods = [
        "fit",
        "create_train_X_y",
        "backtest_refit_parallel",
        "backtest_refit_no_parallel",
        "backtest_no_refit_parallel",
        "backtest_no_refit_no_parallel",
        "gridSearch_refit_parallel",
        "gridSearch_refit_no_parallel"
]


for i, regressor in enumerate(regressors):
    print("")
    print(regressor, param_grids[i])
    print("")
    forecaster = ForecasterAutoreg(regressor=regressor, lags=50)
    
    # Profiling fit
    # ==============================================================================
    start = time.time()
    forecaster.fit(y=y, exog=exog)
    end = time.time()
    elapsed_times.append(end - start)
    
    # Profiling create_train_X_y
    # ==============================================================================
    start = time.time()
    _ = forecaster.create_train_X_y(y=y, exog=exog)
    end = time.time()
    elapsed_times.append(end - start)

    # Profiling backtesting refit and parallel
    # ==============================================================================
    start = time.time()
    metric, backtest_predictions = backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = exog,
                                    initial_train_size  = len(y_train),
                                    fixed_train_size    = False,
                                    steps               = 24,
                                    metric              = 'mean_squared_error',
                                    refit               = True,
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    verbose             = False,
                                    show_progress       = False,
                                    n_jobs              = -1
                            )
    end = time.time()
    elapsed_times.append(end - start)
    
    # Profiling backtesting refit and no parallel
    # ==============================================================================
    start = time.time()
    metric, backtest_predictions = backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = exog,
                                    initial_train_size  = len(y_train),
                                    fixed_train_size    = False,
                                    steps               = 24,
                                    metric              = 'mean_squared_error',
                                    refit               = True,
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    verbose             = False,
                                    show_progress       = False,
                                    n_jobs              = 1
                            )
    end = time.time()
    elapsed_times.append(end - start)
    
    # Profiling backtesting no refit and parallel
    # ==============================================================================
    start = time.time()
    metric, backtest_predictions = backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = exog,
                                    initial_train_size  = len(y_train),
                                    fixed_train_size    = False,
                                    steps               = 24,
                                    metric              = 'mean_squared_error',
                                    refit               = False,
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    verbose             = False,
                                    show_progress       = False,
                                    n_jobs              = -1
                            )
    end = time.time()
    elapsed_times.append(end - start)
    
    # Profiling backtesting no refit and no parallel
    # ==============================================================================
    start = time.time()
    metric, backtest_predictions = backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = exog,
                                    initial_train_size  = len(y_train),
                                    fixed_train_size    = False,
                                    steps               = 24,
                                    metric              = 'mean_squared_error',
                                    refit               = False,
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    verbose             = False,
                                    show_progress       = False,
                                    n_jobs              = 1
                            )
    end = time.time()
    elapsed_times.append(end - start)    
    
    # gridSearch refit parallel
    # ==============================================================================
    start = time.time()
    results_grid = grid_search_forecaster(
                    forecaster          = forecaster,
                    y                   = y,
                    exog                = exog,
                    initial_train_size  = len(y_train),
                    steps               = 24,
                    param_grid         = param_grids[i],
                    lags_grid          = lags_grid,
                    refit              = True,
                    metric             = 'mean_squared_error',
                    fixed_train_size   = False,
                    return_best        = False,
                    verbose            = False,
                    show_progress      = False,
                    n_jobs             = -1
            )
    end = time.time()
    elapsed_times.append(end - start)
    
    # GridSearch refit no parallel
    # ==============================================================================
    start = time.time()
    results_grid = grid_search_forecaster(
                    forecaster          = forecaster,
                    y                   = y,
                    exog                = exog,
                    initial_train_size  = len(y_train),
                    steps               = 24,
                    param_grid         = param_grids[i],
                    lags_grid          = lags_grid,
                    refit              = True,
                    metric             = 'mean_squared_error',
                    fixed_train_size   = False,
                    return_best        = False,
                    verbose            = False,
                    show_progress      = False,
                    n_jobs             = 1
            )
    end = time.time()
    elapsed_times.append(end - start)    

-----------------
ForecasterAutoreg
-----------------

Ridge(random_state=36987) {'alpha': [0.1, 1, 10]}



Number of models compared: 9.
Number of models compared: 9.

RandomForestRegressor(random_state=36987) {'n_estimators': [5], 'max_depth': [5, 10, 15]}



In [ ]:
results_grid = pd.DataFrame({
        "regressor": np.repeat(np.array([regressor.__class__.__name__ for regressor in regressors]), len(methods)),
        "method": np.tile(methods, len(regressors)),
        "elapsed_time": elapsed_times
})


results_grid

,regressor,method,elapsed_time
0,Ridge,fit,0.026493
1,Ridge,create_train_X_y,0.007757
2,Ridge,backtest_refit_parallel,1.088614
3,Ridge,backtest_refit_no_parallel,4.669540
4,Ridge,backtest_no_refit_parallel,0.384801
5,Ridge,backtest_no_refit_no_parallel,0.778063
6,Ridge,gridSearch_refit_parallel,9.189203
7,Ridge,gridSearch_refit_no_parallel,42.395821
8,Ridge,fit,0.023748
9,Ridge,create_train_X_y,0.009606
